<a href="https://colab.research.google.com/github/Addyk-24/SMART-RESEARCH-ANALYST/blob/main/SMART_RESEARCH_ANALYST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st


# Imports for the Libraries

import os
import arxiv
import time
import requests
from typing import List, Dict
from datetime import datetime
from io import BytesIO

# Import key components from langchain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# Import ReportLab for PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image

from colorama import init, Fore, Back, Style






# Setting up the Pdf generating class for saving all data in pdf


class PDFGenerator:
    def __init__(self):
        # Create styles for PDF
        self.styles = getSampleStyleSheet()

        # Create custom styles - using unique names to avoid conflicts
        self.styles.add(ParagraphStyle(
            name='CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=18,
            spaceAfter=12,
            textColor=colors.darkblue
        ))

        self.styles.add(ParagraphStyle(
            name='CustomSubtitle',
            parent=self.styles['Heading2'],
            fontSize=14,
            spaceAfter=8,
            textColor=colors.darkblue
        ))

        self.styles.add(ParagraphStyle(
            name='CustomBody',
            parent=self.styles['Normal'],
            fontSize=11,
            leading=14,
            spaceAfter=6
        ))

        # Create directory for PDF exports
        os.makedirs("exports", exist_ok=True)

    def generate_timestamp(self):
        """Generate a timestamp string for filenames"""
        return datetime.now().strftime("%Y%m%d_%H%M%S")

    def create_search_results_pdf(self, query: str, results: List[Dict]):
        """Generate PDF of search results"""
        filename = f"exports/search_results_{self.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph(f"arXiv Search Results: {query}", self.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}", self.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Results
        for paper in results:
            story.append(Paragraph(f"Paper {paper['id']}: {paper['title']}", self.styles['CustomSubtitle']))
            story.append(Paragraph(f"Authors: {paper.get('authors', 'N/A')}", self.styles['CustomBody']))
            story.append(Paragraph(f"Published: {paper.get('published', 'N/A')}", self.styles['CustomBody']))
            story.append(Paragraph(f"URL: {paper['url']}", self.styles['CustomBody']))
            if 'summary' in paper:
                story.append(Paragraph(f"Summary: {paper['summary']}", self.styles['CustomBody']))
            story.append(Spacer(1, 12))

        # Build PDF
        doc.build(story)
        print(f"Search results saved to {filename}")
        return filename

    def create_qa_pdf(self, paper_title: str, question: str, answer: str):
        """Generate PDF of Q&A interactions"""
        filename = f"exports/qa_response_{self.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph("Research Paper Q&A", self.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Paper: {paper_title}", self.styles['CustomSubtitle']))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}", self.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Q&A Content
        story.append(Paragraph(f"Question:", self.styles['CustomSubtitle']))
        story.append(Paragraph(question, self.styles['CustomBody']))
        story.append(Spacer(1, 12))


        story.append(Paragraph(f"Answer:", self.styles['CustomSubtitle']))
        # Split answer into paragraphs for better formatting
        paragraphs = answer.split('\n\n')
        for para in paragraphs:
            story.append(Paragraph(para, self.styles['CustomBody']))
            story.append(Spacer(1, 6))

        # Build PDF
        doc.build(story)
        print(f"Q&A response saved to {filename}")
        return filename

    def create_summary_pdf(self, paper_title: str, summary: str):
        """Generate PDF of paper summary"""
        filename = f"exports/paper_summary_{self.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph("Paper Summary", self.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Paper: {paper_title}", self.styles['CustomSubtitle']))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}", self.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Summary
        story.append(Paragraph("Summary:", self.styles['CustomSubtitle']))

        # Split summary into paragraphs for better formatting
        paragraphs = summary.split('\n\n')
        for para in paragraphs:
            story.append(Paragraph(para, self.styles['CustomBody']))
            story.append(Spacer(1, 6))

        # Build PDF
        doc.build(story)
        print(f"Paper summary saved to {filename}")
        return filename


### MAIN RESEARCH AGENT

In [ ]:
class ResearchAgent:
    def __init__(self):
        print("Initializing Research Agent...")
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.2)
        self.vector_db = None
        self.pdf_generator = PDFGenerator()
        self.current_paper_title = None
        self.news_retriever = NewsRetriever(self.pdf_generator)
        self.research_coach = ResearchCoachAgent(self.pdf_generator)

        print("Research Agent ready!")

    def search_papers(self, query: str, max_results: int = 5):
        """Search for papers on arXiv based on a query"""
        print(f"Searching for papers about: {query}")
        client = arxiv.Client()
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )

        results = []
        for i, paper in enumerate(client.results(search)):
            print(f"\n--- Paper {i+1} ---")
            print(f"Title: {paper.title}")
            print(f"Authors: {', '.join(author.name for author in paper.authors)}")
            print(f"Published: {paper.published.strftime('%Y-%m-%d')}")
            print(f"URL: {paper.pdf_url}")
            print(f"Summary: {paper.summary[:150]}...")

            results.append({
                "id": i+1,
                "title": paper.title,
                "url": paper.pdf_url,
                "authors": ', '.join(author.name for author in paper.authors),
                "published": paper.published.strftime('%Y-%m-%d'),
                "summary": paper.summary[:150] + "..."
            })

        # Asking if user wants to export results to PDF
        if results:
            # export_choice = input("\nDo you want to save these search results to PDF? (y/n): ")
            export_choice = 'n'
            if export_choice.lower() == 'y':
                self.pdf_generator.create_search_results_pdf(query, results)

        return results

    def process_paper(self, paper_url: str):
        """Download and process a paper for analysis"""
        print(f"\nProcessing paper from: {paper_url}")

        try:
            # Create directory for papers
            os.makedirs("papers", exist_ok=True)

            # Extract paper ID from URL
            paper_id = paper_url.split("/")[-1].replace(".pdf", "")
            file_path = f"papers/{paper_id}.pdf"

            # Download the paper
            print("Downloading paper...")
            client = arxiv.Client()
            paper = next(client.results(arxiv.Search(id_list=[paper_id])))
            paper.download_pdf(dirpath="papers", filename=f"{paper_id}.pdf")
            print("Download completed!")

            # Store the paper title
            self.current_paper_title = paper.title

            # Load the paper
            print("Loading paper content...")
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            print(f"Loaded {len(documents)} pages")

            # Split the documents into chunks
            print("Chunking document...")
            text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
            chunks = text_splitter.split_documents(documents)
            print(f"Created {len(chunks)} chunks")

            # Create the vector store
            print("Creating vector database...")
            self.vector_db = FAISS.from_documents(chunks, self.embeddings)

            print("Paper processed successfully!")
            return True
        except Exception as e:
            print(f"Error processing paper: {e}")
            return False

    def ask_question(self, question: str):
        """Ask a question about the loaded paper"""
        if not self.vector_db:
            print("No paper has been processed yet. Please process a paper first.")
            return

        print(f"\nQuestion: {question}")
        print("Searching for relevant content...")

        # Get relevant chunks
        docs = self.vector_db.similarity_search(question, k=4)

        # Create context from retrieved docs
        context = "\n\n".join([doc.page_content for doc in docs])

        # Generate answer with Gemini LLM
        print("Generating answer...")
        prompt = f"""
        Answer the following question based on the provided context from a research paper.
        If the information is not in the context, Try to answer in general but stick to topic.

        Context:
        {context}

        Question:
        {question}

        Answer:
        """

        response = self.llm.invoke([HumanMessage(content=prompt)])
        answer = response.content

        print("\nAnswer:")
        print(answer)

        # Ask if user wants to export Q&A to PDF
        export_choice = input("\nDo you want to save this Q&A to PDF? (y/n): ")
        if export_choice.lower() == 'y':
            self.pdf_generator.create_qa_pdf(self.current_paper_title or "Unknown Paper", question, answer)

        return answer

    def summarize_paper(self):
        """Generate a summary of the paper"""
        if not self.vector_db:
            print("No paper has been processed yet. Please process a paper first.")
            return

        print("\nGenerating paper summary...")

        # Get a representative sample of the paper
        docs = self.vector_db.similarity_search("This paper is about", k=5)
        context = "\n\n".join([doc.page_content for doc in docs])

        # Create a summary prompt and key concepts for visualization
        # Few-shot prompting
        prompt = f"""
        Provide a concise but comprehensive summary of the research paper based on the
        following excerpts. Focus on the main contributions, methodology, results,
        and conclusions. Convert it into paragraphs like for contributions, methodology,
        result and conculsion.

        First add summary then main conepts After this from research paper Add  main concepts,

        Format your response as:

        CONCEPT 1: [Title]
        [Description]

        CONCEPT 2: [Title] (if applicable)
        [Description] (if applicable)

        Many more concepts if needed, if not then do that

        Paper excerpts:
        {context}

        Summary:
        """

        # Generate summary with Gemini
        response = self.llm.invoke([HumanMessage(content=prompt)])
        summary = response.content

        print("\nPaper Summary:")
        print(summary)


        # Ask if user wants to export summary to PDF
        export_choice = input("\nDo you want to save this summary to PDF? (y/n): ")
        if export_choice.lower() == 'y':
            self.pdf_generator.create_summary_pdf(self.current_paper_title or "Unknown Paper", summary)

        return summary

    def extract_citations(self):
        """Extract citation information from the paper"""
        if not self.vector_db:
            print("No paper has been processed yet. Please process a paper first.")
            return

        print("\nExtracting citation information...")

        # Search for sections likely to contain citations
        docs = self.vector_db.similarity_search("references citations bibliography", k=5)
        context = "\n\n".join([doc.page_content for doc in docs])

        # Create a citations extraction prompt
        prompt = f"""
        Extract and format the key citations from the following sections of a research paper.
        If you can identify them, provide the citations in a structured format.

        Paper excerpts:
        {context}

        Citations:
        """

        # Generate citations with Gemini
        response = self.llm.invoke([HumanMessage(content=prompt)])
        citations = response.content


        print("\nCitation Information:")
        print(citations)

        # Ask if user wants to export citations to PDF
        export_choice = input("\nDo you want to save these citations to PDF? (y/n): ")
        if export_choice.lower() == 'y':
            # Create a custom PDF for citations
            filename = f"exports/paper_citations_{self.pdf_generator.generate_timestamp()}.pdf"
            doc = SimpleDocTemplate(filename, pagesize=letter)
            story = []

            # Title
            story.append(Paragraph("Paper Citations", self.pdf_generator.styles['CustomTitle']))
            story.append(Spacer(1, 12))
            story.append(Paragraph(f"Paper: {self.current_paper_title or 'Unknown Paper'}",
                                  self.pdf_generator.styles['CustomSubtitle']))
            story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
                                  self.pdf_generator.styles['CustomBody']))
            story.append(Spacer(1, 24))

            # Citations
            story.append(Paragraph("Extracted Citations:", self.pdf_generator.styles['CustomSubtitle']))

            # Split citations for better formatting
            citation_list = citations.split('\n')
            for citation in citation_list:
                if citation.strip():
                    story.append(Paragraph(citation, self.pdf_generator.styles['CustomBody']))
                    story.append(Spacer(1, 6))

            # Build PDF
            doc.build(story)
            print(f"Citations saved to {filename}")

        return citations


### NEWS RETRIEVEL

In [ ]:
class NewsRetriever:
    """Class to retrieve and process news articles related to a research topic"""

    def __init__(self, pdf_generator):
        self.pdf_generator = pdf_generator
        # Initialize Google Gemini LLM
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
        # NewsAPI key
        self.news_api_key = os.environ.get("NEWS_API_KEY")

    def get_latest_news(self, topic, max_results=5):
        """Fetch latest news articles on a given topic"""
        print(f"\nFetching latest news about: {topic}")

        try:
            # Prepare the API request to NewsAPI
            url = "https://newsapi.org/v2/everything"
            params = {
                "q": topic,
                "sortBy": "publishedAt",
                "language": "en",
                "pageSize": max_results,
                "apiKey": self.news_api_key
            }

            # Make the request
            response = requests.get(url, params=params)

            if response.status_code != 200:
                print(f"Error fetching news: {response.status_code}")
                return []

            data = response.json()
            articles = data.get("articles", [])

            # Process and display the results
            results = []
            for i, article in enumerate(articles):
                print(f"\n--- News {i+1} ---")
                print(f"Title: {article['title']}")
                print(f"Source: {article['source']['name']}")
                print(f"Published: {article['publishedAt']}")
                print(f"URL: {article['url']}")
                print(f"Description: {article['description']}")

                results.append({
                    "id": i+1,
                    "title": article['title'],

                    "source": article['source']['name'],

                    "published": article['publishedAt'],
                    "url": article['url'],
                    "description": article['description']
                })

            # Ask if user wants to export results to PDF
            if results:
                export_choice = input("\nDo you want to save these news results to PDF? (y/n): ")
                if export_choice.lower() == 'y':
                    self._create_news_pdf(topic, results)

                # Ask if user wants to analyze the news
                analyze_choice = input("\nDo you want to analyze how these news relate to research trends? (y/n): ")
                if analyze_choice.lower() == 'y':
                    self._analyze_news_trends(topic, results)

            return results

        except Exception as e:
            print(f"Error retrieving news: {e}")
            return []

    def _create_news_pdf(self, topic, news_results):
        """Generate PDF of news results"""
        filename = f"exports/news_results_{self.pdf_generator.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph(f"Latest News: {topic}", self.pdf_generator.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
                             self.pdf_generator.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Results

        for article in news_results:
            story.append(Paragraph(f"Article {article['id']}: {article['title']}",
                                 self.pdf_generator.styles['CustomSubtitle']))
            story.append(Paragraph(f"Source: {article['source']}", self.pdf_generator.styles['CustomBody']))
            story.append(Paragraph(f"Published: {article['published']}", self.pdf_generator.styles['CustomBody']))
            story.append(Paragraph(f"URL: {article['url']}", self.pdf_generator.styles['CustomBody']))
            story.append(Paragraph(f"Description: {article['description']}", self.pdf_generator.styles['CustomBody']))
            story.append(Spacer(1, 12))

        # Build PDF
        doc.build(story)
        print(f"News results saved to {filename}")
        return filename

    def _analyze_news_trends(self, topic, news_results):
        """Analyze news articles using Gemini to identify research trends"""
        print("\nAnalyzing news articles for research trends...")

        # Prepare content for analysis
        articles_text = ""
        for article in news_results:
            articles_text += f"Title: {article['title']}\n"
            articles_text += f"Description: {article['description']}\n\n"

        # Create prompt for analysis
        prompt = f"""
        Analyze these recent news articles about "{topic}" and identify:
        1. Key research trends or developments
        2. Emerging questions or challenges in this field
        3. How these news items relate to current academic research

        News articles:
        {articles_text}

        Analysis:
        """

        # Generate analysis with Gemini
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            analysis = response.content

            print("\nTrend Analysis:")
            print(analysis)

            # Ask if user wants to export analysis to PDF
            export_choice = input("\nDo you want to save this analysis to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_analysis_pdf(topic, news_results, analysis)

            return analysis

        except Exception as e:
            print(f"Error analyzing news: {e}")
            return "Analysis could not be completed due to an error."

    def _create_analysis_pdf(self, topic, news_results, analysis):
        """Generate PDF of news analysis"""
        filename = f"exports/news_analysis_{self.pdf_generator.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph(f"News Analysis: {topic}", self.pdf_generator.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
                             self.pdf_generator.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Articles analyzed
        story.append(Paragraph("Articles Analyzed:", self.pdf_generator.styles['CustomSubtitle']))
        for article in news_results:
            story.append(Paragraph(f"• {article['title']} ({article['source']})",
                                 self.pdf_generator.styles['CustomBody']))
        story.append(Spacer(1, 12))

        # Analysis
        story.append(Paragraph("Trend Analysis:", self.pdf_generator.styles['CustomSubtitle']))

        # Split analysis into paragraphs for better formatting
        paragraphs = analysis.split('\n\n')
        for para in paragraphs:
            if para.strip():
                story.append(Paragraph(para, self.pdf_generator.styles['CustomBody']))
                story.append(Spacer(1, 6))

        # Build PDF
        doc.build(story)
        print(f"News analysis saved to {filename}")
        return filename


### RESEARCH COACH

In [ ]:
class ResearchCoachAgent:
    """Agent to provide guidance on academic writing, publishing, and research best practices"""

    def __init__(self, pdf_generator):
        self.pdf_generator = pdf_generator
        # Initialize Google Gemini LLM
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.2)
        # Store common writing and publishing advice topics
        self.advice_topics = {
            "paper_structure": "Structure and organization of academic papers",
            "writing_style": "Academic writing style and best practices",
            "journal_selection": "How to select appropriate journals for publication",
            "submission_process": "The paper submission and review process",
            "addressing_reviews": "How to address reviewer comments",
            "collaboration": "Research collaboration strategies",
            "literature_review": "How to conduct an effective literature review",
            "methodology": "Research methodology and experimental design",
            "data_visualization": "Effective data visualization in research papers",
            "citations": "Citation practices and reference management",
            "impact_factor": "Understanding journal impact factors"
        }

    def get_writing_advice(self, topic=None):
        """Get advice on writing research papers"""
        if not topic:

            # Display available topics and let user choose
            print("\nAvailable Writing Advice Topics:")
            for i, (key, description) in enumerate(self.advice_topics.items(), 1):
                print(f"{i}. {description}")

            try:
                choice = int(input("\nSelect a topic number (or 0 for custom topic): "))
                if choice == 0:
                    topic = input("Enter your custom topic: ")
                else:
                    topic = list(self.advice_topics.keys())[choice-1]
            except (ValueError, IndexError):
                topic = input("Enter your specific writing question: ")

        print(f"\nGenerating advice on: {topic}")

        # Create advice prompt for the LLM
        prompt = f"""
        As an expert academic writing coach, provide comprehensive advice on the following topic:

        Topic: {topic}

        Please include:
        1. Best practices for this aspect of academic writing
        2. Common mistakes to avoid
        3. Practical examples where appropriate
        4. Specific tips for different research fields if relevant
        5. Resources that might be helpful


        Make your advice actionable, clear, and suitable for researchers at any stage of their career.
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            advice = response.content

            print("\nWriting Advice:")
            print(advice)

            # Ask if user wants to export advice to PDF
            export_choice = input("\nDo you want to save this advice to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Writing Advice", topic, advice)

            return advice

        except Exception as e:
            print(f"Error generating advice: {e}")
            return "Could not generate advice due to an error."

    def get_publishing_guidance(self, field=None):
        """Get guidance on publishing in specific academic fields"""
        if not field:
            field = input("\nEnter your research field (e.g., Computer Science, Biochemistry): ")

        print(f"\nGenerating publishing guidance for: {field}")

        # Create publishing guidance prompt for the LLM
        prompt = f"""
        As an expert in academic publishing, provide comprehensive guidance on publishing research papers in {field}.

        Please include:
        1. Top journals and conferences in {field} (without providing specific journal names if you're not certain)
        2. Typical submission and review timelines
        3. Key factors that increase chances of acceptance
        4. Common reasons for rejection
        5. Open access vs. traditional publishing considerations
        6. Preprint strategies
        7. Tips for addressing reviewer comments

        Make your guidance specific to {field} where possible, and provide practical, actionable advice.
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            guidance = response.content

            print("\nPublishing Guidance:")
            print(guidance)

            # Ask if user wants to export guidance to PDF
            export_choice = input("\nDo you want to save this publishing guidance to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Publishing Guidance", field, guidance)

            return guidance

        except Exception as e:
            print(f"Error generating publishing guidance: {e}")
            return "Could not generate publishing guidance due to an error."

    def analyze_paper_draft(self, draft_text):
        """Analyze a paper draft and provide improvement suggestions"""
        if not draft_text or len(draft_text) < 100:
            print("Please provide a substantial draft text for analysis.")
            return

        print("\nAnalyzing paper draft...")

        # Create Analyzing paper prompt for the LLM
        prompt = f"""
        As an expert academic editor, analyze the following paper draft and provide constructive feedback.

        Focus on:
        1. Structure and organization
        2. Clarity and coherence
        3. Academic writing style
        4. Argumentation and logical flow
        5. Potential areas for improvement

        For each area, provide specific suggestions with examples from the text where possible.

        Paper draft:
        {draft_text[:4000]}  # Limit to first 4000 chars to stay within token limits

        Analysis:
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            analysis = response.content

            print("\nDraft Analysis:")
            print(analysis)

            # Ask if user wants to export analysis to PDF
            export_choice = input("\nDo you want to save this draft analysis to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Paper Draft Analysis", "Draft Review", analysis)

            return analysis

        except Exception as e:
            print(f"Error analyzing draft: {e}")
            return "Could not analyze draft due to an error."

    def get_research_timeline(self, project_type=None):
        """Generate a research project timeline with milestones"""
        if not project_type:
            project_type = input("\nEnter the type of research project (e.g., PhD thesis, journal article, conference paper): ")

        print(f"\nGenerating research timeline for: {project_type}")

        # Prompt
        prompt = f"""
        Create a detailed research timeline with milestones for a {project_type}.

        Include:
        1. Major phases of the research process
        2. Key milestones and deliverables
        3. Estimated time requirements for each phase
        4. Critical checkpoints and decision points
        5. Tips for staying on schedule

        Format the timeline in a clear, structured manner with specific timeframes where appropriate.
        Make the timeline realistic and adaptable for researchers at different career stages.
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            timeline = response.content

            print("\nResearch Timeline:")
            print(timeline)

            # Ask if user wants to export timeline to PDF
            export_choice = input("\nDo you want to save this research timeline to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Research Timeline", project_type, timeline)

            return timeline

        except Exception as e:
            print(f"Error generating timeline: {e}")
            return "Could not generate timeline due to an error."

    def answer_research_question(self, question):
        """Answer specific questions about research, writing, or publishing"""
        print(f"\nAnswering research question: {question}")

        # Create prompt for the LLM
        prompt = f"""
        As an expert in academic research, writing, and publishing, answer the following question
        with detailed, practical, and accurate information:

        Question: {question}

        Provide a comprehensive answer that addresses all aspects of the question.
        Include examples, best practices, and actionable advice where appropriate.
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            answer = response.content

            print("\nAnswer:")
            print(answer)

            # Ask if user wants to export answer to PDF
            export_choice = input("\nDo you want to save this answer to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Research Question", question, answer)

            return answer

        except Exception as e:
            print(f"Error answering question: {e}")
            return "Could not answer the question due to an error."

    def generate_paper_outline(self, topic, paper_type=None):
        """Generate a structured outline for a research paper"""
        if not paper_type:
            paper_type = input("\nEnter paper type (e.g., original research, review, case study): ")

        print(f"\nGenerating {paper_type} paper outline for: {topic}")

        # Create prompt for the LLM
        prompt = f"""
        Create a detailed outline for a {paper_type} paper on the topic: {topic}

        Include:
        1. Title suggestions
        2. Abstract structure
        3. Main sections and subsections with brief descriptions
        4. Key points to address in each section
        5. Suggestions for figures or tables if appropriate

        Format the outline with clear hierarchical structure and follow standard academic conventions
        for {paper_type} papers. Make the outline comprehensive yet adaptable.
        """

        # Get response from LLM
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            outline = response.content

            print("\nPaper Outline:")
            print(outline)

            # Ask if user wants to export outline to PDF
            export_choice = input("\nDo you want to save this paper outline to PDF? (y/n): ")
            if export_choice.lower() == 'y':
                self._create_advice_pdf("Paper Outline", topic, outline)

            return outline

        except Exception as e:
            print(f"Error generating outline: {e}")
            return "Could not generate outline due to an error."

    def _create_advice_pdf(self, advice_type, topic, content):
        """Generate PDF of research advice"""
        filename = f"exports/{advice_type.lower().replace(' ', '_')}_{self.pdf_generator.generate_timestamp()}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title
        story.append(Paragraph(f"{advice_type}: {topic}", self.pdf_generator.styles['CustomTitle']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
                             self.pdf_generator.styles['CustomBody']))
        story.append(Spacer(1, 24))

        # Content - split by headings and paragraphs for better formatting
        content_parts = content.split('\n\n')
        for part in content_parts:
            if part.strip():
                # Check if this part looks like a heading
                if len(part.strip()) < 100 and (part.strip().endswith(':') or part.strip().isupper() or
                                               any(part.strip().startswith(h) for h in ['#', 'I.', 'II.', '1.', '2.'])):
                    story.append(Paragraph(part.strip(), self.pdf_generator.styles['CustomSubtitle']))
                else:
                    story.append(Paragraph(part.strip(), self.pdf_generator.styles['CustomBody']))

                story.append(Spacer(1, 6))

        # Build PDF
        doc.build(story)
        print(f"{advice_type} saved to {filename}")
        return filename


## MAIN FUNCTION


In [ ]:
# Main Function


def main():
    """Main function to run the Streamlit app"""
    st.set_page_config(page_title="Smart Research Analyst", page_icon=":book:", layout="wide")

    # App headerclear

    st.title("Smart Research Analyst")
    st.write("Your AI-powered research assistant for academic writing, publishing, and analysis.")

    # Sidebar for settings and navigation
    st.sidebar.header("Settings")

    # API Key management
    google_api_key = st.sidebar.text_input("Gemini API Key", type="password")
    news_api_key = st.sidebar.text_input("NewsAPI Key", type="password")

    if google_api_key and news_api_key:
        os.environ["GOOGLE_API_KEY"] = google_api_key
        os.environ["NEWS_API_KEY"] = news_api_key
        st.sidebar.success("API keys set successfully!")

        # Initialize agents
        agent = ResearchAgent()

        # Main navigation
        st.sidebar.header("Navigation")
        menu_options = ["Search Papers", "Process Papers", "Latest News", "Research Coach"]
        selected_menu = st.sidebar.selectbox("Select a function", menu_options)

        # Handle different navigation options
        if selected_menu == "Search Papers":
            search_papers_ui(agent)
        elif selected_menu == "Process Papers":
            process_papers_ui(agent)
        elif selected_menu == "Latest News":
            latest_news_ui(agent)
        elif selected_menu == "Research Coach":
            research_coach_ui(agent)
    else:
        st.sidebar.warning("Please enter your API keys to use the app.")
        st.info("This application requires Gemini API and NewsAPI keys to function. Please enter your API keys in the sidebar.")


def search_papers_ui(agent):
    """UI for searching papers"""
    st.header("Search Latest Research Papers")

    with st.form("search_form"):
        query = st.text_input("Enter your research topic:")
        max_results = st.slider("Maximum number of results", 1, 20, 5)
        submit_search = st.form_submit_button("Search Papers")

    if submit_search and query:
        with st.spinner("Searching for papers..."):
            papers = agent.search_papers(query, max_results)

        if papers:
            st.success(f"Found {len(papers)} papers related to '{query}'")

            # Display results in an expandable section
            for i, paper in enumerate(papers):
                with st.expander(f"{i+1}. {paper['title']}"):
                    st.write(f"**Authors:** {paper['authors']}")
                    st.write(f"**Published:** {paper['published']}")
                    st.write(f"**Summary:** {paper['summary']}")
                    st.markdown(f"[View Paper]({paper['url']})")

            # Option to save results
            if st.button("Save Results to PDF"):
                with st.spinner("Generating PDF..."):
                    pdf_path = agent.pdf_generator.create_search_results_pdf(query, papers)
                    st.success(f"Results saved to {pdf_path}")
        else:
            st.warning("No papers found for the given query. Try a different search term.")


def process_papers_ui(agent):
    """UI for processing papers and asking questions"""
    st.header("Process Paper & Analysis")

    paper_url = st.text_input("Enter arXiv paper URL (e.g., https://arxiv.org/pdf/2201.12345.pdf):")

    col1, col2 = st.columns(2)
    with col1:
        process_btn = st.button("Process Paper")

    if process_btn and paper_url:
        with st.spinner("Processing paper... This may take a minute."):
            success = agent.process_paper(paper_url)

        if success:
            st.session_state.paper_processed = True
            st.success("Paper processed successfully!")
        else:
            st.error("Failed to process paper. Check the URL and try again.")

    # Only show these options if a paper has been processed
    if st.session_state.get('paper_processed', False):
        st.subheader("Paper Analysis Tools")

        analysis_tab1, analysis_tab2, analysis_tab3 = st.tabs([
            "Ask Questions", "Generate Summary", "Extract Citations"
        ])

        with analysis_tab1:
            with st.form("question_form"):
                question = st.text_input("Ask a question about the paper:")
                submit_question = st.form_submit_button("Get Answer")

            if submit_question and question:
                with st.spinner("Generating answer..."):
                    answer = agent.ask_question(question)
                st.markdown("### Answer")
                st.write(answer)
                if st.button("Save Q&A to PDF"):
                    pdf_path = agent.pdf_generator.create_qa_pdf(
                        agent.current_paper_title or "Unknown Paper",
                        question,
                        answer
                    )
                    st.success(f"Q&A saved to {pdf_path}")

        with analysis_tab2:
            if st.button("Generate Paper Summary"):
                with st.spinner("Generating summary..."):
                    summary = agent.summarize_paper()
                st.markdown("### Paper Summary")
                st.write(summary)
                if st.button("Save Summary to PDF"):
                    pdf_path = agent.pdf_generator.create_summary_pdf(
                        agent.current_paper_title or "Unknown Paper",
                        summary
                    )
                    st.success(f"Summary saved to {pdf_path}")

        with analysis_tab3:
            if st.button("Extract Citations"):
                with st.spinner("Extracting citations..."):
                    citations = agent.extract_citations()
                st.markdown("### Citations")
                st.write(citations)


def latest_news_ui(agent):
    """UI for getting latest news"""
    st.header("Latest Research News")

    with st.form("news_form"):
        topic = st.text_input("Enter research topic for news:")
        max_news = st.slider("Maximum number of news articles", 1, 10, 5)
        submit_news = st.form_submit_button("Get Latest News")

    if submit_news and topic:
        with st.spinner("Fetching latest news..."):
            news_results = agent.news_retriever.get_latest_news(topic, max_news)

        if news_results:
            st.success(f"Found {len(news_results)} news articles about '{topic}'")

            for i, article in enumerate(news_results):
                with st.expander(f"{i+1}. {article['title']}"):
                    st.write(f"**Source:** {article['source']}")
                    st.write(f"**Published:** {article['published']}")
                    st.write(f"**Description:** {article['description']}")
                    st.markdown(f"[Read Article]({article['url']})")

            col1, col2 = st.columns(2)
            with col1:
                if st.button("Save News to PDF"):
                    pdf_path = agent.news_retriever._create_news_pdf(topic, news_results)
                    st.success(f"News saved to {pdf_path}")

            with col2:
                if st.button("Analyze Research Trends"):
                    with st.spinner("Analyzing trends..."):
                        analysis = agent.news_retriever._analyze_news_trends(topic, news_results)
                    st.markdown("### Trend Analysis")
                    st.write(analysis)
        else:
            st.warning("No news found for the given topic. Try a different search term.")


def research_coach_ui(agent):
    """UI for research coach functionality"""
    st.header("Research Coach")

    coach_options = [
        "Publishing Guidance",
        "Writing Advice",
        "Research Timeline",
        "Paper Draft Analysis",
        "Paper Outline",
        "Research Question"
    ]

    selected_option = st.radio("Select coaching service:", coach_options)

    if selected_option == "Publishing Guidance":
        with st.form("publishing_form"):
            field = st.text_input("Enter your research field (e.g., Computer Science, Biology):")
            submit_field = st.form_submit_button("Get Publishing Guidance")

        if submit_field and field:
            with st.spinner("Generating publishing guidance..."):
                guidance = agent.research_coach.get_publishing_guidance(field)
            st.markdown("### Publishing Guidance")
            st.write(guidance)
            if st.button("Save to PDF"):
                pdf_path = agent.research_coach._create_advice_pdf("Publishing Guidance", field, guidance)
                st.success(f"Guidance saved to {pdf_path}")

    elif selected_option == "Writing Advice":
        topics = list(agent.research_coach.advice_topics.values())
        selected_topic = st.selectbox("Select writing advice topic:", topics)

        if st.button("Get Writing Advice"):
            with st.spinner("Generating writing advice..."):
                advice = agent.research_coach.get_writing_advice(selected_topic)
            st.markdown("### Writing Advice")
            st.write(advice)
            if st.button("Save to PDF"):
                pdf_path = agent.research_coach._create_advice_pdf("Writing Advice", selected_topic, advice)
                st.success(f"Advice saved to {pdf_path}")

    elif selected_option == "Research Timeline":
        with st.form("timeline_form"):
            project_type = st.text_input("Enter project type (e.g., PhD thesis, conference paper):")
            submit_project = st.form_submit_button("Generate Timeline")

        if submit_project and project_type:
            with st.spinner("Generating research timeline..."):
                timeline = agent.research_coach.get_research_timeline(project_type)
            st.markdown("### Research Timeline")
            st.write(timeline)
            if st.button("Save to PDF"):
                pdf_path = agent.research_coach._create_advice_pdf("Research Timeline", project_type, timeline)
                st.success(f"Timeline saved to {pdf_path}")

    elif selected_option == "Paper Draft Analysis":
        with st.form("draft_form"):
            draft_text = st.text_area("Paste your paper draft here:", height=300)
            submit_draft = st.form_submit_button("Analyze Draft")

        if submit_draft and draft_text:
            if len(draft_text) < 100:
                st.warning("Please provide a longer draft for meaningful analysis.")
            else:
                with st.spinner("Analyzing paper draft..."):
                    analysis = agent.research_coach.analyze_paper_draft(draft_text)
                st.markdown("### Draft Analysis")
                st.write(analysis)
                if st.button("Save to PDF"):
                    pdf_path = agent.research_coach._create_advice_pdf("Paper Draft Analysis", "Draft Review", analysis)
                    st.success(f"Analysis saved to {pdf_path}")

    elif selected_option == "Paper Outline":
        with st.form("outline_form"):
            topic = st.text_input("Enter paper topic:")
            paper_type = st.text_input("Enter paper type (e.g., original research, review):")
            submit_outline = st.form_submit_button("Generate Outline")

        if submit_outline and topic and paper_type:
            with st.spinner("Generating paper outline..."):
                outline = agent.research_coach.generate_paper_outline(topic, paper_type)
            st.markdown("### Paper Outline")
            st.write(outline)
            if st.button("Save to PDF"):
                pdf_path = agent.research_coach._create_advice_pdf("Paper Outline", topic, outline)
                st.success(f"Outline saved to {pdf_path}")

    elif selected_option == "Research Question":
        with st.form("question_form"):
            question = st.text_input("Enter your research question:")
            submit_question = st.form_submit_button("Get Answer")

        if submit_question and question:
            with st.spinner("Generating answer..."):
                answer = agent.research_coach.answer_research_question(question)
            st.markdown("### Answer")
            st.write(answer)
            if st.button("Save to PDF"):
                pdf_path = agent.research_coach._create_advice_pdf("Research Question", question, answer)
                st.success(f"Answer saved to {pdf_path}")


# Initialize session state when app loads
if 'paper_processed' not in st.session_state:
    st.session_state.paper_processed = False


if __name__ == "__main__":
    main()